In [36]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [37]:
# BASIC CONFIG (필요한 파라미터 값 여기서 설정)

data_dir = '/content/drive/MyDrive/Deep_learning_Team_PJ/hand_tracing/yolo_eraser/data/'
model = 'yolov8n.pt'  # pretrained yolov8n

classes = ['blue_eraser', 'red_eraser']  # 라벨명
yaml_file = 'eraser_data.yaml'  # yaml 파일명

training_epoch = 20
patience = 5  # 지정한 epoch 만큼 성능이 개선이 안되면 조기종료.
batch = 16  # batch size (dafault)
imgsz = 640  # input batch size (default)
optimizer = 'Adam'


In [26]:
!nvidia-smi

Tue Dec 12 13:32:56 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P0    27W /  70W |   1557MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### ULTRALYTICS SETUP

In [5]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.227 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 27.0/166.8 GB disk)


In [ ]:
from ultralytics import YOLO
import os
import yaml
from IPython.display import display, Image
from IPython import display

display.clear_output()
!yolo checks

In [7]:
# install comet
%pip install comet_ml --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.7/586.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 514.7/514.7 kB 26.1 MB/s eta 0:00:00


In [9]:
#@title Select YOLOv8 🚀 logger {run: 'auto'}
logger = 'Comet' #@param ['Comet', 'TensorBoard']

# if logger == 'Comet':
#   %pip install -q comet_ml
#   import comet_ml; comet_ml.init()
# elif logger == 'TensorBoard':
#   %load_ext tensorboard
#   %tensorboard --logdir .

### Create yaml file

In [38]:
# python: dictionary <----------> yaml
data = {
    'train': data_dir + 'train/',
    'val': data_dir + 'val/',
    'test': data_dir + 'test/',
    'nc': 2,
    'names': classes
}

os.makedirs(data_dir, exist_ok = True) # directory to save

with open(data_dir + yaml_file, 'wt') as fw:
    yaml.dump(data, fw)


In [39]:
with open(data_dir + yaml_file, 'rt') as fr:
    d = yaml.safe_load(fr)
    print(type(d))
    print(d)

<class 'dict'>
{'names': ['blue_eraser', 'red_eraser'], 'nc': 2, 'test': '/content/drive/MyDrive/Deep_learning_Team_PJ/hand_tracing/yolo_eraser/data/test/', 'train': '/content/drive/MyDrive/Deep_learning_Team_PJ/hand_tracing/yolo_eraser/data/train/', 'val': '/content/drive/MyDrive/Deep_learning_Team_PJ/hand_tracing/yolo_eraser/data/val/'}


### Model Training

In [40]:
model = YOLO(model)  # pretrained yolov8n

model.train(data = data_dir + yaml_file,
            epochs = training_epoch,
            patience = patience, # 지정한 epoch 만큼 성능이 개선이 안되면 조기종료.
            batch = batch, # batch size (dafault)
            imgsz = imgsz, # input batch size (default)
            optimizer = optimizer)

       4/20      2.33G      1.379      1.403      1.317         64        640: 100%|██████████| 10/10 [00:13<00:00,  1.34s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.38it/s]

                   all         40         80   0.000824      0.138   0.000494   0.000194



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       5/20      2.33G      1.367      1.348      1.402         40        640: 100%|██████████| 10/10 [00:09<00:00,  1.04it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.82it/s]

                   all         40         80     0.0118        0.2    0.00479    0.00122


...

                   all         40         80      0.996      0.959      0.985      0.741



20 epochs completed in 0.098 hours.
Optimizer stripped from runs/detect/train5/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train5/weights/best.pt, 6.2MB

Validating runs/detect/train5/weights/best.pt...
Ultralytics YOLOv8.0.227 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.15it/s]


                   all         40         80      0.996      0.959      0.985      0.739
           blue_eraser         40         40      0.991       0.95      0.977      0.719
            red_eraser         40         40          1      0.968      0.993      0.758
Speed: 0.2ms preprocess, 3.2ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs/detect/train5


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x77fa94d83c70>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

### Model Evaluation

In [41]:
best_model_path = '/content/runs/detect/train5/weights/best.pt'

best_model = YOLO(best_model_path)
metrics = best_model.val()

for label, ap in zip(classes, metrics.box.maps):
    print(label, ':', ap)

Ultralytics YOLOv8.0.227 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/drive/MyDrive/Deep_learning_Team_PJ/hand_tracing/yolo_eraser/data/val/labels.cache... 40 images, 0 backgrounds, 0 corrupt: 100%|██████████| 40/40 [00:00<?, ?it/s]

val: WARNING ⚠️ /content/drive/MyDrive/Deep_learning_Team_PJ/hand_tracing/yolo_eraser/data/val/images/image_151.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /content/drive/MyDrive/Deep_learning_Team_PJ/hand_tracing/yolo_eraser/data/val/images/image_157.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /content/drive/MyDrive/Deep_learning_Team_PJ/hand_tracing/yolo_eraser/data/val/images/image_160.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /content/drive/MyDrive/Deep_learning_Team_PJ/hand_tracing/yolo_eraser/data/val/images/image_167.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /content/drive/MyDrive/Deep_learning_Team_PJ/hand_tracing/yolo_eraser/data/val/images/image_175.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /content/drive/MyDrive/Deep_learning_Team_PJ/hand_tracing/yolo_eraser/data/val/images/image_179.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /content/drive/MyDrive/Deep_learning_Team_PJ/hand_tracing/yolo_eraser/data/val/images/imag


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:08<00:00,  2.81s/it]


                   all         40         80      0.985      0.952      0.986       0.74
           blue_eraser         40         40      0.995       0.95      0.979      0.715
            red_eraser         40         40      0.974      0.954      0.992      0.766
Speed: 0.6ms preprocess, 31.2ms inference, 0.0ms loss, 3.0ms postprocess per image
Results saved to runs/detect/val
blue_eraser : 0.7146497593610983
red_eraser : 0.7657895882762414
